In [0]:
# Download the data from Google Drive, and parse it to some Python lists
import gdown
import csv

def load_data_from_drive():
    
    edges = []
    gdown.download("https://drive.google.com/uc?id=1vBUOcQVh1EDY91xtW4fZjowbN8RFQ5Br", "interactions.txt", quiet=True)
    with open("interactions.txt") as f:
        c = csv.reader(f, delimiter="\t")
        for row in c:
            # TODO: Are weights minutes or what?
            edges.append(tuple(int(i) for i in row))

    people = []
    jobs = []
    gdown.download("https://drive.google.com/uc?id=10e8tFloUoUpV8UUf863blJBk8eTrSUih", "person_descriptions.txt", quiet=True)
    with open("person_descriptions.txt") as f:
        c = csv.reader(f, delimiter="\t")
        for row in c:
            people.append(int(row[0]))
            jobs.append(row[1])
    
    return edges, people, jobs


In [0]:
class Graphs:
    def adjacent_edges(self, nodes, halfk):
        n = len(nodes)
        for i, u in enumerate(nodes):
            for j in range(i+1, i+halfk+1):
                v = nodes[j % n]
                yield (u, v, 5.0)
    def make_ring_lattice(self, n, k):
        G = nx.Graph()
        nodes = range(n)
        G.add_nodes_from(nodes)
        edges = list(self.adjacent_edges(nodes, k//2))
        G.add_weighted_edges_from(edges)
        return G
    def make_ws_graph(self, n, k, p):
        def flip(p):
            return np.random.random() < p
        G = self.make_ring_lattice(n, k)
        nodes = set(G)
        for u, v in G.edges():
            if flip(p):
                choices = nodes - {u} - set(G[u])
                new_v = np.random.choice(list(choices))
                G.remove_edge(u, v)
                G.add_edge(u, new_v, weight = 5.0)
        return G    
    

In [0]:
class Graphs:
    def adjacent_edges(self, nodes, halfk):
        n = len(nodes)
        for i, u in enumerate(nodes):
            for j in range(i+1, i+halfk+1):
                v = nodes[j % n]
                yield u, v
    def make_ring_lattice(self, n, k):
        G = nx.Graph()
        nodes = range(n)
        G.add_nodes_from(nodes)
        G.add_edges_from(self.adjacent_edges(nodes, k//2))
        return G
    def make_ws_graph(self, n, k, p):
        def flip(p):
            return np.random.random() < p
        G = self.make_ring_lattice(n, k)
        nodes = set(G)
        for u, v in G.edges():
            if flip(p):
                choices = nodes - {u} - set(G[u])
                new_v = np.random.choice(list(choices))
                G.remove_edge(u, v)
                G.add_edge(u, new_v)
        return G    

In [0]:
class School(Graphs):
    def __init__(self, n=20, k = 4, p = .5):
        super()
        self.G = self.make_ws_graph(n, k, p)
        initialState= {"state": "susceptible"}
        attributes = {i:deepcopy(initialState) for i in self.G.nodes}
        nx.set_node_attributes(self.G, attributes)
        
    def get_neighbors_weights(self, node):
        arr = []
        for n in self.G.neighbors(node):
            arr.append( (n, self.G.edges[(node, n)]["weight"]) )
        return arr
    
    def expose(self, index):
        self.G.node[index]["state"] = "exposed"
        
    def infect(self, index):
        self.G.node[index]["state"] = "infectious"
        
    def recover(self, index):
        self.G.node[index]["state"] = "recovered"
        
    def get_colors(self):
        def color(state):
            if state == "susceptible":
                return "green"
            elif state == "exposed":
                return "yellow"
            if state == "infectious":
                return "red"
            if state == "recovered":
                return "blue"
        states = list([data["state"] for i, data in self.G.nodes(data=True)])
        return list(map(color, states))
    
    def get_global_state(self):
        globalState = {}
        for index, attributes in self.G.nodes(data=True):
            state = attributes["state"]
            globalState[state] = globalState.get(state, 0) + 1

        return globalState
    
    def visualize(self):
        nx.draw_networkx(self.G, 
                node_color=self.get_colors(), 
                node_size=500, 
                with_labels=True)
        print(self.get_global_state())
        

In [0]:
S = School()
S.infect(0)

In [0]:
S.visualize()